In [356]:
from models.experimental import attempt_load
import torch
from utils.torch_utils import TracedModel
device = torch.device('cpu')
weights = 'yolov7-tiny-custom.pt'
model = attempt_load(weights, map_location=device)  # load FP32 model

Fusing layers... 


In [170]:
from torchinfo import summary
summary(model, (1, 3, 640, 640))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [1, 25200, 13]            --
├─Sequential: 1-1                        --                        --
│    └─Conv: 2-1                         [1, 32, 320, 320]         --
│    │    └─Conv2d: 3-1                  [1, 32, 320, 320]         896
│    │    └─LeakyReLU: 3-2               [1, 32, 320, 320]         --
│    └─Conv: 2-2                         [1, 64, 160, 160]         --
│    │    └─Conv2d: 3-3                  [1, 64, 160, 160]         18,496
│    │    └─LeakyReLU: 3-4               [1, 64, 160, 160]         --
│    └─Conv: 2-3                         [1, 32, 160, 160]         --
│    │    └─Conv2d: 3-5                  [1, 32, 160, 160]         2,080
│    │    └─LeakyReLU: 3-6               [1, 32, 160, 160]         --
│    └─Conv: 2-4                         [1, 32, 160, 160]         --
│    │    └─Conv2d: 3-7                  [1, 32, 160, 160]         2,080
│   

In [290]:
print(model)

Model(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): Conv(
      (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (3): Conv(
      (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (4): Conv(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (5): Conv(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (6): Concat()
    (7): Conv(
      (conv): Conv2d(1

In [357]:
from tqdm.auto import tqdm
import numpy as np


for i in tqdm(range(len(model.model))):
    try:
        model.model[i].conv
    except AttributeError:
        continue
    # try:
    #     u, s, v = torch.svd_lowrank(model.model[i].conv.weight, 2)
    # except:
    shape = model.model[i].conv.weight.shape
    size = shape[0]*shape[1]*shape[2]*shape[3]
    for n in range(int(np.sqrt(size)), 0, -1):
        if (size%n == 0):
            break
    rank = n//16 if model.model[i].conv.weight.norm().detach().numpy() < 4 else n
    # rank = 32
    try:
        u, s, v = torch.svd_lowrank(model.model[i].conv.weight.reshape(-1, n), rank)
    except AssertionError:
        try:
            u, s, v = torch.svd_lowrank(model.model[i].conv.weight.reshape(-1, n), rank//2)
        except AssertionError:
            u, s, v = torch.svd_lowrank(model.model[i].conv.weight.reshape(-1, n), rank//4)
    model.model[i].conv.weight = torch.nn.Parameter(torch.matmul(torch.matmul(u, torch.diag_embed(s)), v.mT).reshape(shape).half())


  0%|          | 0/78 [00:00<?, ?it/s]

In [358]:
model.float()

/Users/yewon/opt/anaconda3/envs/diffusion/lib/python3.9/site-packages/torch/_tensor.py:1104: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1646756029501/work/build/aten/src/ATen/core/TensorBody.h:475.)
  return self._grad


Model(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): Conv(
      (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (3): Conv(
      (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (4): Conv(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (5): Conv(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (6): Concat()
    (7): Conv(
      (conv): Conv2d(1

In [359]:
import torch.nn.utils.prune as prune
from tqdm.auto import tqdm


for i in tqdm(range(len(model.model))):
    try:
        model.model[i].conv
    except AttributeError:
        continue
    model.model[i].conv.weight = torch.nn.Parameter(model.model[i].conv.weight)

parameters_to_prune = []
for layer in model.model:
    try:
        parameters_to_prune.append((layer.conv, 'weight'))
    except AttributeError:
        continue

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.95,
)
for i in tqdm(range(len(model.model))):
    try:
        model.model[i].conv
    except AttributeError:
        continue
    prune.remove(model.model[i].conv, 'weight')

  0%|          | 0/78 [00:00<?, ?it/s]

  0%|          | 0/78 [00:00<?, ?it/s]

In [360]:
print(list(model.named_parameters())[1])

('model.0.conv.weight', Parameter containing:
tensor([[[[  0.00000,  -0.52490,  -0.00000],
          [ -1.52051,  -1.11719,  -0.36157],
          [  0.20032,  -0.81348,  -0.00000]],

         [[ -0.92627,  -0.25781,   0.93799],
          [ -2.91797,  -1.39648,   0.50439],
          [  0.53174,   0.30054,   1.22559]],

         [[ -0.32837,  -0.00000,   0.38379],
          [ -1.81934,  -0.47144,   0.32178],
          [  0.21265,   0.35474,   0.28711]]],


        [[[ -1.85547,   0.13403,  -0.41650],
          [  0.91309,   4.15234,   4.53516],
          [ -0.17322,  -3.96289,  -3.38086]],

         [[ -2.09766,   0.00000,  -0.93115],
          [  1.84961,   5.51953,   5.19531],
          [ -0.42651,  -5.03516,  -4.38672]],

         [[ -1.22754,  -0.54834,  -0.92188],
          [  0.47754,   2.27344,   1.87793],
          [  0.00000,  -1.72949,  -1.39746]]],


        [[[  1.83691,   0.77539,  -2.51172],
          [  1.17090,   4.57031,  -5.81250],
          [  1.83984,   3.41992,  -5.9

In [361]:
print(
    "Sparsity in conv1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.model[2].conv.weight == 0))
        / float(model.model[2].conv.weight.nelement())
    )
)

Sparsity in conv1.weight: 57.67%


In [285]:
model

Model(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): Conv(
      (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (3): Conv(
      (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (4): Conv(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (5): Conv(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (6): Concat()
    (7): Conv(
      (conv): Conv2d(1

In [363]:
from copy import deepcopy, copy
ckpt = {'model': deepcopy(model).half(),
}
# Save last, best and delete
torch.save(ckpt, 'yolov7-svd_reshape-prune.pt')